In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://heartspringhealth.com/wp-content/uploads/2014/05/cars-c.jpg", width=1000, height=250)

The Motor Vehicle Collisions crash table contains details on the crash event. Each row represents a crash event. The Motor Vehicle Collisions data tables contain information from all police reported motor vehicle collisions in NYC. The police report (MV104-AN) is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage

Below are the steps to perform our ETL processes:
1. Extract data daily from NYC Open Data (https://opendata.cityofnewyork.us/) via Socrata API
2. For our initial load, we will only be using data from 2021 - now to populate our table
3. For our daily load, we will perform upsert functions against our data to only inserting newer data to the table
4. Data will be stored in the Amazon S3 bucket
5. We then proceed to perform necessary transformations then load to our Data Warehouse in AWS SQL Server 

In [2]:
#Install required libraries
!pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Import required Libraries 
import pandas as pd 
from sodapy import Socrata 
from geopy.geocoders import Nominatim

#**Data Extraction**#

In [36]:
client = Socrata("data.cityofnewyork.us", None)

# Get 1000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("h9gi-nx95", order='crash_date DESC', limit=2000)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_date                     2000 non-null   object
 1   crash_time                     2000 non-null   object
 2   latitude                       1831 non-null   object
 3   longitude                      1831 non-null   object
 4   location                       1831 non-null   object
 5   on_street_name                 1451 non-null   object
 6   number_of_persons_injured      2000 non-null   object
 7   number_of_persons_killed       2000 non-null   object
 8   number_of_pedestrians_injured  2000 non-null   object
 9   number_of_pedestrians_killed   2000 non-null   object
 10  number_of_cyclist_injured      2000 non-null   object
 11  number_of_cyclist_killed       2000 non-null   object
 12  number_of_motorist_injured     2000 non-null   object
 13  num

In [38]:
#See all the columns in the dataframe
pd.set_option('display.max_columns', None)

In [39]:
#Check all the column names 
df.columns

Index(['crash_date', 'crash_time', 'latitude', 'longitude', 'location',
       'on_street_name', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'contributing_factor_vehicle_1',
       'contributing_factor_vehicle_2', 'collision_id', 'vehicle_type_code1',
       'off_street_name', 'vehicle_type_code2', 'borough', 'zip_code',
       'cross_street_name', 'contributing_factor_vehicle_3',
       'vehicle_type_code_3', 'contributing_factor_vehicle_4',
       'vehicle_type_code_4', 'contributing_factor_vehicle_5',
       'vehicle_type_code_5'],
      dtype='object')

In [44]:
sorted_df = df.sort_values(by=['crash_date', 'crash_time'], ascending=True)

In [45]:
sorted_df = sorted_df[['crash_date', 'crash_time', 'number_of_persons_injured', 'number_of_persons_killed', 'number_of_pedestrians_injured',              
                      'number_of_pedestrians_killed', 'number_of_cyclist_injured', 'number_of_cyclist_killed', 'number_of_motorist_injured',                             
                      'number_of_motorist_killed', 'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2', 'vehicle_type_code1', 
                      'vehicle_type_code2', 'latitude', 'longitude', 'on_street_name', 'borough', 'zip_code',]]

In [46]:
df.head()

,crash_date,crash_time,latitude,longitude,location,on_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,off_street_name,vehicle_type_code2,borough,zip_code,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2022-07-15T00:00:00.000,8:30,40.685806,-73.773155,"{'latitude': '40.685806', 'longitude': '-73.77...",119 AVENUE,0,0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,4546931,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-07-15T00:00:00.000,8:55,40.63552,-74.0167,"{'latitude': '40.63552', 'longitude': '-74.016...",65 STREET,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4546392,Pick-up Truck,6 AVENUE,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-15T00:00:00.000,4:00,40.674397,-73.896515,"{'latitude': '40.674397', 'longitude': '-73.89...",PENNSYLVANIA AVENUE,1,0,0,0,0,0,1,0,Unspecified,NaN,4546522,Station Wagon/Sport Utility Vehicle,LIBERTY AVENUE,NaN,BROOKLYN,11207,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-07-15T00:00:00.000,14:19,40.760944,-73.9163,"{'latitude': '40.760944', 'longitude': '-73.91...",41 STREET,1,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,4546590,Station Wagon/Sport Utility Vehicle,31 AVENUE,E-Scooter,QUEENS,11103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-07-15T00:00:00.000,13:40,40.6953,-73.812096,"{'latitude': '40.6953', 'longitude': '-73.8120...",VANWYCK EXPRESSWAY,1,0,0,0,0,0,1,0,Unspecified,NaN,4546871,Station Wagon/Sport Utility Vehicle,97 AVENUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
sorted_df.head()

,crash_date,crash_time,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,latitude,longitude,on_street_name,borough,zip_code
1972,2022-07-06T00:00:00.000,0:35,0,0,0,0,0,0,0,0,Unsafe Lane Changing,Unspecified,Tractor Truck Diesel,Station Wagon/Sport Utility Vehicle,40.730442,-73.91367,LONG ISLAND EXPRESSWAY,NaN,NaN
1929,2022-07-06T00:00:00.000,10:00,0,0,0,0,0,0,0,0,Unspecified,NaN,Sedan,NaN,40.751854,-74.002396,NaN,MANHATTAN,10001
1932,2022-07-06T00:00:00.000,10:02,0,0,0,0,0,0,0,0,Unspecified,NaN,Garbage or Refuse,NaN,40.869843,-73.91588,BROADWAY,NaN,NaN
1991,2022-07-06T00:00:00.000,10:15,1,0,0,0,1,0,0,0,Illnes,NaN,Bike,NaN,NaN,NaN,NaN,NaN,NaN
1941,2022-07-06T00:00:00.000,10:25,1,0,0,0,0,0,1,0,Unspecified,NaN,Station Wagon/Sport Utility Vehicle,NaN,40.668648,-73.92835,EASTERN PARKWAY,NaN,NaN


In [53]:
#Use geopy library to get more address information based on the coordinate points (longitude, latitude)

# initialize Nominatim API|
geolocator = Nominatim(user_agent="geoapiExercises")

In [80]:
# Latitude & Longitude input 
latitude = sorted_df.loc[3, 'latitude']
longitude = sorted_df.loc[3, 'longitude']

location = geolocator.reverse(latitude+","+longitude)

address = location.raw['address']

print(address)

# traverse the data
street_name = address.get('road')
zipcode = address.get('postcode')
neighbourhood = address.get('neighbourhood')
borough = address.get('suburb')

print('street_name : ', street_name)
print('zip_code : ', zipcode)
print('neighbourhood : ', neighbourhood)
print('Borough : ', borough)

{'house_number': '30-97', 'road': '41st Street', 'neighbourhood': 'Sunnyside Gardens', 'suburb': 'Queens', 'city': 'City of New York', 'state': 'New York', 'ISO3166-2-lvl4': 'US-NY', 'postcode': '11103', 'country': 'United States', 'country_code': 'us'}
street_name :  41st Street
zip_code :  11103
neighbourhood :  Sunnyside Gardens
Borough :  Queens


In [76]:
latitude = []
longitude = []
street_name = []
zipcode = []
neighbourhood = []
borough = []

for i in range(0,len(sorted_df)):
  lat = sorted_df.loc[i, 'latitude']
  long = sorted_df.loc[i, 'longitude']

  latitude.append(lat)
  longitude.append(long)

  location = geolocator.reverse(lat+","+long)

  address = location.raw['address']

  street_name.append(address.get('road'))
  zipcode.append(address.get('postcode'))
  neighbourhood.append(address.get('neighbourhood'))
  borough.append(address.get('suburb'))

TypeError: ignored